# API Center ❤️ all APIs

## Import from IBM® API Connect

![image](../../images/import-from-ibm.png)

Playground to experiment bulk importing APIs from IBM API Connect. We start by creating an API Center instance using [Bicep resource definition](https://learn.microsoft.com/en-us/azure/templates/microsoft.apicenter/services?pivots=deployment-language-bicep) and then we will use the [IBM API Connect Tooklit](https://www.ibm.com/docs/en/api-connect/10.0.8?topic=tool-overview-command-line) to export the APIs and the [Azure CLI](https://learn.microsoft.com/en-us/cli/azure/apic?view=azure-cli-latest) to import the APIs into API Center.

💡 Every step outlined below leverages the power of PowerShell scripts. This is designed so you can seamlessly integrate them into your automation workflows, CI/CD pipelines, webhooks, and more.

### Learning Objectives - Upon completing this lab, you should be able to

- Deploy Azure API Center using Bicep.
- Configure the API Center metadata scheme for APIs, Environments and Deployments.
- Understand the Azure CLI commands to manage Azure API Center. [Full list of commands available here](https://learn.microsoft.com/en-us/cli/azure/apic?view=azure-cli-latest).
- Register APIs using the OpenAPI spec exported from IBM API Connect.
- Create deployments for each IBM API Connect Gateway.
- Search and discover the APIs registered in API Center.

### Prerequisites

- Install or update to the latest [.NET 8 SDK](https://dotnet.microsoft.com/en-us/download)
- Install or update to the latest [Visual Studio Code](https://code.visualstudio.com/)
- Install the [Polyglot Notebooks extension](https://marketplace.visualstudio.com/items?itemName=ms-dotnettools.dotnet-interactive-vscode) from the VS Code marketplace
- Install or update to the latest [PowerShell](https://learn.microsoft.com/en-us/powershell/scripting/install/installing-powershell)
- Install or update to the latest [Azure CLI](https://learn.microsoft.com/en-us/cli/azure/install-azure-cli). The Azure API Center extension will automatically install the first time you run an [az apic](https://learn.microsoft.com/en-us/cli/azure/apic?view=azure-cli-latest) command.
- [An Azure Subscription](https://azure.microsoft.com/en-us/free/) with Contributor permissions
- [Sign in to Azure with Azure CLI](https://learn.microsoft.com/en-us/cli/azure/authenticate-azure-cli-interactively)
- Install or update to the latest [IBM API Connect Toolkit](https://www.ibm.com/docs/en/api-connect/10.0.8?topic=configuration-installing-toolkit)
- [Logging in to a management server](https://www.ibm.com/docs/en/api-connect/10.0.8?topic=tool-logging-in-management-server)


### 0️⃣ Initialize notebook variables

- Resources will be suffixed by a unique string based on your subscription id
- Adjust the APIC location parameter according your preferences and [region availability.](https://learn.microsoft.com/en-us/azure/api-center/overview#available-regions) 


In [ ]:

$deploymentName = Split-Path -Path (Get-Location) -Leaf
$resourceGroupName = "lab-$deploymentName" # change the name to match your naming style
$resourceGroupLocation = "uksouth"
$apicResourceNamePrefix = "apic1"
$apicResourceSku = "free"
$apicResourceTags = @{
  "lab" = $deploymentName
}

$toolkitManagementServer = "platform-api.eu-west-a.apiconnect.automation.ibm.com"

# the following metadata will be created during the deployment
$metadata = @(
  @{
    "name" = 'orgId'
    "schema" = '{"title":"orgId","description":"Organization Id","type":"string"}'
    "assignedTo" = @(
      @{
        "entity" = 'environment'
        "required" = $false
      }
    )
  },
  @{
    "name" = 'region'
    "schema" = '{"title":"region","description":"Environment region","type":"string"}'
    "assignedTo" = @(
      @{
        "entity" = 'environment'
        "required" = $false
      }
    )
  },
  @{
    "name" = 'accountId'
    "schema" = '{"title":"accountId","description":"Account Id","type":"string"}'
    "assignedTo" = @(
      @{
        "entity" = 'environment'
        "required" = $false
      }
    )
  },
  @{
    "name" = 'tenantOwner'
    "schema" = '{"title":"tenantOwner","description":"Tenant Owner Email","type":"string","format":"email"}'
    "assignedTo" = @(
      @{
        "entity" = 'environment'
        "required" = $false
      }
    )
  },
  @{
    "name" = 'createdDate'
    "schema" = '{"title":"createdDate","description":"Date when the API was created","type":"string"}'
    "assignedTo" = @(
      @{
        "entity" = 'environment'
        "required" = $false
      },
      @{
        "entity" = 'api'
        "required" = $false
      }
    )
  },
  @{
    "name" = 'Catalog'
    "schema" = '{"title":"Catalog","description":"API Connect Catalog","type":"string"}'
    "assignedTo" = @(
      @{
        "entity" = 'api'
        "required" = $false
      }
    )
  },
  @{
    "name" = 'Product'
    "schema" = '{"title":"Product","description":"API Connect Product","type":"string"}'
    "assignedTo" = @(
      @{
        "entity" = 'api'
        "required" = $false
      }
    )
  },
  @{
    "name" = 'Plan'
    "schema" = '{"title":"Plan","description":"API Connect Plan","type":"string"}'
    "assignedTo" = @(
      @{
        "entity" = 'api'
        "required" = $false
      }
    )
  },
  @{
    "name" = 'ManagementURL'
    "schema" = '{"title":"ManagementURL","description":"API Connect API URL","type":"string"}'
    "assignedTo" = @(
      @{
        "entity" = 'api'
        "required" = $false
      }
    )
  },
  @{
    "name" = 'State'
    "schema" = '{"title":"State","description":"Publishing state","type":"string"}'
    "assignedTo" = @(
      @{
        "entity" = 'api'
        "required" = $false
      }
    )
  },
  @{
    "name" = 'Tags'
    "schema" = '{"title":"Tags","description":"API tags","type":"array","items":{"type":"string"}}'
    "assignedTo" = @(
      @{
        "entity" = 'api'
        "required" = $false
      }
    )
  },
  @{
    "name" = 'GatewayType'
    "schema" = '{"title":"GatewayType","description":"Type of API Connect Gateway where the API is deployed","type":"string"}'
    "assignedTo" = @(
      @{
        "entity" = 'deployment'
        "required" = $false
      }
    )
  },
  @{
    "name" = 'GatewayServiceUrls'
    "schema" = '{"title":"GatewayServiceUrls","description":"Gateway Service Urls","type":"string"}'
    "assignedTo" = @(
      @{
        "entity" = 'deployment'
        "required" = $false
      }
    )
  }
)

# the following environment(s) will be created during the deployment
$environments = @(
)

Write-Output "✅ Variables initialized ⌚ $(Get-Date -Format 'HH:mm:ss')"


### 1️⃣ Create the Azure Resource Group
All resources deployed in this lab will be created in the specified resource group. Skip this step if you want to use an existing resource group.

In [ ]:
$resourceGroupOutput = az group create --name $resourceGroupName --location $resourceGroupLocation

if ($LASTEXITCODE -ne 0) {
    Write-Output $resourceGroupOutput
} else {
    Write-Output "✅ Azure Resource Grpup $resourceGroupName created ⌚ $(Get-Date -Format 'HH:mm:ss')"
}

### 2️⃣ Create deployment using 🦾 Bicep

This lab uses [Bicep](https://learn.microsoft.com/en-us/azure/azure-resource-manager/bicep/overview?tabs=bicep) to declarative define all the resources that will be deployed. Change the parameters or the [main.bicep](main.bicep) directly to try different configurations. 

In [ ]:
$bicepParameters = @{
    "`$schema" = "https://schema.management.azure.com/schemas/2019-04-01/deploymentParameters.json#"
    "contentVersion" = "1.0.0.0"
    "parameters" = @{
        "apicResourceNamePrefix" = @{ "value" = $apicResourceNamePrefix }
        "apicResourceTags" = @{ "value" = $apicResourceTags }
        "apicMetadataSchema" = @{ "value" = $metadata }
        "apicEnvironments" = @{ "value" = $environments }
    }
}

$bicepParametersJson = ConvertTo-Json -InputObject $bicepParameters -Depth 10
Set-Content -Path "params.json" -Value $bicepParametersJson

# Execute the Azure CLI command to create the deployment
az deployment group create --name $deploymentName --resource-group $resourceGroupName --template-file "main.bicep" --parameters "params.json"



### 3️⃣ Get the deployment outputs

We will set the `apicResourceName` variable with the value that was returned from the deployment 

In [ ]:
$deploymentOutput = az deployment group show --name $deploymentName -g $resourceGroupName --query properties.outputs.apicResourceName.value -o json
if ($LASTEXITCODE -ne 0) {
    Write-Output $deploymentOutput
} else {
    $apicResourceName = $deploymentOutput | ConvertFrom-Json
}
Write-Output "👉🏻 API Center name: $apicResourceName"

### 4️⃣ Create the API Center environments based on each API Connect org

In [ ]:

$orgsJson = apic orgs -s $toolkitManagementServer --format json --my

$orgs = $orgsJson | ConvertFrom-Json
foreach ($org in $orgs.results) {
    $orgName = $org.name

    $catalogsJson = apic catalogs --org $orgName -s $toolkitManagementServer --format json
    $catalogs = $catalogsJson | ConvertFrom-Json
    foreach ($catalog in $catalogs.results) {
        $catalogName = $catalog.name
        $catalogTitle = $catalog.title
        $catalogDescription = $catalog.summary

        $environmentCustomProperties = @{
            "orgId" = $org.id
            "region" = $org.metadata.region
            "accountId" = $org.metadata.accountId
            "tenantOwner" = $org.metadata.tenant_owner_email
            "createdDate" = $org.metadata.created_at
        }
        $environmentCustomPropertiesJson = ConvertTo-Json -InputObject $environmentCustomProperties -Depth 10 -Compress
        Write-Output "👉🏻 Creating environment $catalogName"
        az apic environment create -g $resourceGroupName -n $apicResourceName --environment-id $catalogName --title $catalogTitle `
                --description $catalogDescription --type $catalogTitle --custom-properties $environmentCustomPropertiesJson.replace('"','\"') `
                --server "{'type': 'Azure API Management'}" --onboarding "{'developerPortalUri': []}"
    }

}



### 5️⃣ Export the APIs from IBM API Connect and Register them in API Center



In [ ]:
rm -Recurse .temp
mkdir .temp

$orgsJson = apic orgs -s $toolkitManagementServer --format json --my

$orgs = $orgsJson | ConvertFrom-Json
foreach ($org in $orgs.results) {
    $environmentName = $org.title
    $orgName = $org.name
    $catalogsJson = apic catalogs --org $orgName -s $toolkitManagementServer --format json
    $catalogs = $catalogsJson | ConvertFrom-Json
    foreach ($catalog in $catalogs.results) {
        $catalogName = $catalog.name
        $productsJson = apic products --org $orgName -s $toolkitManagementServer --scope catalog -c $catalogName --format json
        $products = $productsJson | ConvertFrom-Json
        foreach ($product in $products.results) {
            $productName = $product.name
            foreach ($plan in $product.plans) {
                $planName = $plan.name
                foreach ($api in $plan.apis) {
                    $apiID = $api.id
                    $apiDetailsJson = apic apis:list --org $orgName -s $toolkitManagementServer --scope catalog -c $catalogName --format json $apiID
                    $apiDetails = $apiDetailsJson | ConvertFrom-Json
                    $apiName = $apiDetails.name
                    $apiVersion = $apiDetails.version
                    $apiVersionName = $apiVersion.replace('.', '-')
                    $apiNameAndVersion = $apiName + ":" + $apiVersion
                    $apiSpecFile = $apiName + "_" + $apiVersion + ".json"
 
                    apic apis:get --org $orgName -s $toolkitManagementServer --scope catalog -c $catalogName --format json $apiNameAndVersion --output .temp

                    write-output "👉🏻 Registering API $apiName"
                    az apic api register -g $resourceGroupName -n $apicResourceName -l .temp/$apiSpecFile

                    $apiCustomProperties = @{ 
                        "createdDate" = $apiDetails.created_at      
                        "Catalog" = $catalogName
                        "Product" = $productName
                        "Plan" = $planName
                        "ManagementURL" = $apiDetails.url
                        "State" = $apiDetails.state
                    }
                    $apiCustomPropertiesJson = ConvertTo-Json -InputObject $apiCustomProperties -Depth 10 -Compress
                    az apic api update -g $resourceGroupName -n $apicResourceName --api-id $apiName --custom-properties $apiCustomPropertiesJson.replace('"','\"')    

                    $openAPIJson = Get-Content -Raw .temp/$apiSpecFile
                    $openAPI = $openAPIJson | ConvertFrom-Json
#                    $runtimeUri = $openAPI.servers[0].url

                        
                    $deploymentId = $org.title
                    # apic gateway-services --org $orgName -s $toolkitManagementServer --format json --scope org
                    $server = $('{\"runtimeUri\":[\"https://api.eu-west-a.apiconnect.ibmappdomain.cloud\"]}')

                    $deploymentCustomProperties = @{
                        "createdDate" = $apiDetails.createdDate
                        "GatewayType" = $apiDetails.gateway_type
                        "GatewayServiceUrls" = $apiDetails.url
                    }
                    $deploymentCustomPropertiesJson = ConvertTo-Json -InputObject $deploymentCustomProperties -Depth 10 -Compress
                    write-output "👉🏻 Creating $catalogName deployment for API $apiName"
                    az apic api deployment create -g $resourceGroupName -n $apicResourceName --deployment-id $catalogName `
                            --title $catalogName --description $catalogName --api-id $apiName `
                            --environment-id "/workspaces/default/environments/$catalogName" `
                            --definition-id "/workspaces/default/apis/$apiName/versions/$apiVersionName/definitions/openapi" `
                            --server $server `
                            --custom-properties $deploymentCustomPropertiesJson.replace('"','\"')        

                }

            }
        }

    }


}

### Discover the APIs that were just imported

You can discover the APIs with fhe following methods:
- With the Azure Portal
- With the [self-hosted API Center Portal](https://learn.microsoft.com/en-us/azure/api-center/enable-api-center-portal)
- With the [VS Code extension](https://learn.microsoft.com/en-us/azure/api-center/use-vscode-extension-copilot) that is integrated with GitHub Copilot Chat.
- With the CLI, the service REST API and more

Here we will use the [list command](https://learn.microsoft.com/en-us/cli/azure/apic/api?view=azure-cli-latest#az-apic-api-list) to display the APIs that we have just imported. 


In [ ]:
az apic api list -g $resourceGroupName -n $apicResourceName --query "[].{Name:name, Title:title, Kind:kind, ContactEmail:contacts[0].email}" -o table

### 🗑️ Clean up resources

When you're finished with the lab, you should remove all your deployed resources from Azure to avoid extra charges and keep your Azure subscription uncluttered.
Use the [clean-up-resources notebook](clean-up-resources.ipynb) for that.